In [35]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from statsmodels.tsa.stattools import adfuller 
from sklearn.preprocessing import MinMaxScaler 
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM 


import os
import sys
import warnings
warnings.filterwarnings('ignore')
sns.set_style("darkgrid")
%matplotlib inline

In [12]:
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from file_handler import FileHandler
from log import get_logger  
file_handler = FileHandler()
my_logger = get_logger("LSTM_Modeling")  

In [13]:
train_data = file_handler.read_csv("../data/cleaned_train.csv") 

2022-05-28 06:11:08,541 — FileHandler — DEBUG — file read as csv


### Isolate the Rossmann Store Sales Dataset into Time Series Data

In [14]:
data = train_data.groupby("Date").agg({"Sales": "mean"})
data.head(10)

,Sales
Date,
2013-01-01,84.395871
2013-01-02,6178.517489
2013-01-03,5660.173094
2013-01-04,5923.138117
2013-01-05,5299.049327
2013-01-06,119.357848
2013-01-07,9377.999103
2013-01-08,7742.737220
2013-01-09,6887.256502


### Check if the Time Series Data is Stationary

In [15]:
adfResult = adfuller(data.Sales.values, autolag='AIC')
print(f'ADF Statistic: {adfResult[0]}')
print(f'p-value: {adfResult[1]}')

ADF Statistic: -5.751282394418175
p-value: 5.958430360162239e-07


Since the p-value is less than 0.05, the data is not Stationary.

In [16]:
scaler = MinMaxScaler()

try:
    scaler.fit(data.Sales.values.reshape([-1, 1]))
    scaled_array = scaler.transform(data.Sales.values.reshape(-1, 1))
    data['SalesScaled'] = scaled_array
    data.tail(10)

    my_logger.debug("Data scaled successfully.")

except Exception as e:
    my_logger.exception(f"Scaling error, {e}")

2022-05-28 06:11:09,300 — LSTM_Modeling — DEBUG — Data scaled successfully.
2022-05-28 06:11:09,300 — LSTM_Modeling — DEBUG — Data scaled successfully.
2022-05-28 06:11:09,300 — LSTM_Modeling — DEBUG — Data scaled successfully.
2022-05-28 06:11:09,300 — LSTM_Modeling — DEBUG — Data scaled successfully.


In [17]:
# Check if Scaled Sales is Stationary

adfResult = adfuller(data.SalesScaled.values, autolag='AIC')
print(f'ADF Statistic: {adfResult[0]}')
print(f'p-value: {adfResult[1]}')

ADF Statistic: -5.7512823944181894
p-value: 5.958430360161802e-07


### Differencing the Time Series Data

In [18]:
# Create a differenced series
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return pd.Series(diff)

salesScaledDiff = difference(data.SalesScaled.values)
salesScaledDiff

0      0.506975
1     -0.043121
2      0.021876
3     -0.051918
4     -0.430903
         ...   
936    0.760631
937   -0.108573
938   -0.043412
939    0.022197
940    0.089605
Length: 941, dtype: float64

### Making the Data ready for Modeling

In [19]:
SIZE = len(data.SalesScaled)
WINDOW_SIZE = 48
BATCH_SIZE = SIZE - WINDOW_SIZE * 2
EPOCHS = 200

In [20]:
DateTrain = data.index.values[0:BATCH_SIZE]
DateValid = data.index.values[BATCH_SIZE:]
XTrain = data.SalesScaled.values[0:BATCH_SIZE].astype('float32')
XValid = data.SalesScaled.values[BATCH_SIZE:].astype('float32')

# Obtain shapes for vectors of size (,1) for dates series

DateTrain = np.reshape(DateTrain, (-1, 1))
DateValid = np.reshape(DateValid, (-1, 1))

print("Shape of the training set date series: ", DateTrain.shape)
print("Shape of the validation set date series: ", DateValid.shape)
print()
print("Shape of the training set logarithm of sales series: ", XTrain.shape)
print("Shape of the validation set logarithm of sales series in a stateless LSTM: ", XValid.shape)

Shape of the training set date series:  (846, 1)
Shape of the validation set date series:  (96, 1)

Shape of the training set logarithm of sales series:  (846,)
Shape of the validation set logarithm of sales series in a stateless LSTM:  (96,)


In [21]:
tf.random.set_seed(1234)
# add extra dimension
series = tf.expand_dims(XTrain, axis=-1)
series.shape

TensorShape([846, 1])

In [22]:
# create tensor from each individual element
dataset = tf.data.Dataset.from_tensor_slices(series)
dataset

<TensorSliceDataset element_spec=TensorSpec(shape=(1,), dtype=tf.float32, name=None)>

In [23]:
# takes a window_size + 1 chunk from the slices
dataset = dataset.window(WINDOW_SIZE + 1, shift=1, drop_remainder=True)

In [24]:
datasetEx = tf.data.Dataset.from_tensor_slices(tf.range(10))
datasetEx = datasetEx.window(5, shift=1, drop_remainder=True)
for window in datasetEx:
    print([elem.numpy() for elem in window])

[0, 1, 2, 3, 4]
[1, 2, 3, 4, 5]
[2, 3, 4, 5, 6]
[3, 4, 5, 6, 7]
[4, 5, 6, 7, 8]
[5, 6, 7, 8, 9]


In [25]:
dataset = dataset.flat_map(lambda window: window.batch(WINDOW_SIZE + 1))
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
dataset = dataset.batch(BATCH_SIZE).prefetch(1)

In [26]:
def windowed_dataset(series, window_size=WINDOW_SIZE, batch_size=BATCH_SIZE):
  series = tf.expand_dims(series, axis=-1)
  dataset = tf.data.Dataset.from_tensor_slices(series)
  dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
  dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
  dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
  dataset = dataset.batch(batch_size).prefetch(1)
  return dataset

In [27]:
DatasetTrain = windowed_dataset(XTrain)
DatasetVal = windowed_dataset(XValid)

### Model Building

In [28]:
model = Sequential()
model.add(LSTM(8, input_shape=[None, 1], return_sequences=True))
model.add(LSTM(4, input_shape=[None, 1]))
model.add(Dense(1))
model.compile(loss="huber_loss", optimizer='adam')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, None, 8)           320       
                                                                 
 lstm_1 (LSTM)               (None, 4)                 208       
                                                                 
 dense (Dense)               (None, 1)                 5         
                                                                 
Total params: 533
Trainable params: 533
Non-trainable params: 0
_________________________________________________________________


### Training the Model

In [29]:
history = model.fit(DatasetTrain, epochs=EPOCHS, validation_data=DatasetVal, verbose=1)

Epoch 1/200
1/1 [==============================] - 6s 6s/step - loss: 0.2148 - val_loss: 0.2269
Epoch 2/200
1/1 [==============================] - 0s 158ms/step - loss: 0.2083 - val_loss: 0.2202
Epoch 3/200
1/1 [==============================] - 0s 155ms/step - loss: 0.2020 - val_loss: 0.2137
Epoch 4/200
1/1 [==============================] - 0s 153ms/step - loss: 0.1959 - val_loss: 0.2074
Epoch 5/200
1/1 [==============================] - 0s 156ms/step - loss: 0.1900 - val_loss: 0.2012
Epoch 6/200
1/1 [==============================] - 0s 168ms/step - loss: 0.1843 - val_loss: 0.1953
Epoch 7/200
1/1 [==============================] - 0s 145ms/step - loss: 0.1788 - val_loss: 0.1896
Epoch 8/200
1/1 [==============================] - 0s 160ms/step - loss: 0.1735 - val_loss: 0.1840
Epoch 9/200
1/1 [==============================] - 0s 160ms/step - loss: 0.1683 - val_loss: 0.1786
Epoch 10/200
1/1 [==============================] - 0s 161ms/step - loss: 0.1633 - val_loss: 0.1734
Epoch 11/200

In [32]:
from time import gmtime, strftime

time = strftime("%Y-%m-%d", gmtime())

In [33]:
model.save(f'../models/LSTM_sales_prediction_model {time}.pkl')

INFO:tensorflow:Assets written to: ../models/LSTM_sales_prediction_model 2022-05-28.pkl\assets


INFO:tensorflow:Assets written to: ../models/LSTM_sales_prediction_model 2022-05-28.pkl\assets
